In [7]:
import os

In [9]:
os.chdir("x:\DL\Projects\mlops")
%pwd

'x:\\DL\\Projects\\mlops'

In [10]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data: Path
    unzip_dir: Path


In [11]:
from scripts.MLOPs.constants import *
from scripts.MLOPs.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_dataingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data= config.local_data,
            unzip_dir= config.unzip_dir
            )
        return data_ingestion_config
    

In [14]:
import sys,zipfile,os
from urllib import request
from scripts.MLOPs.exception import AppException
from scripts.MLOPs import logger

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    

    def download_data(self)->str:
        try:
            dataset_url = self.config.source_URL
            os.makedirs(self.config.root_dir, exist_ok= True)
            logger.info(f"Downloading data from {dataset_url} into file {self.config.local_data}")
            if not os.path.exists(self.config.local_data):
                request.urlretrieve(
                    url = dataset_url, filename=self.config.local_data
                )
                logger.info(f"{self.config.local_data} downloaded!!")
            else:
                logger.info(f"File already exists on {self.config.local_data}")
            
        
        except Exception as e:
            raise AppException(e, sys)
    
    def extract_zipfile(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data, 'r') as z:
            z.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    dataingestion_config = config.get_dataingestion_config()
    dataingestion = DataIngestion(config=dataingestion_config)
    dataingestion.download_data()
    dataingestion.extract_zipfile()
except Exception as eL:
    raise AppException(eL, sys)

[2024-07-28 03:05:52,947: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2024-07-28 03:05:52,950: INFO: common: yaml file: params.yaml loaded sucessfully]
[2024-07-28 03:05:52,950: INFO: common: created directory at artifacts]
[2024-07-28 03:05:52,950: INFO: common: created directory at artifacts/data_ingestion]
[2024-07-28 03:05:52,956: INFO: 2195735928: Downloading data from https://universe.roboflow.com/ds/i02h52wl6q?key=ZM0MEFLZ5R into file artifacts/data_ingestion/data.zip\data.zip]


KeyboardInterrupt: 